In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [2]:
# ============================================================
# STAGE 1 v5 — CLEAN MINIMAL FOUNDATION
# Payment/Service switch • No lag drop • No exposure drift
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"

klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# =============================
# CLEAN COLUMNS
# =============================

def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

klaim = klaim.drop_duplicates().reset_index(drop=True)
polis = polis.drop_duplicates().reset_index(drop=True)

# =============================
# DATE PARSING
# =============================

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = pd.to_datetime(klaim[col], errors="coerce")

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = pd.to_datetime(polis[col], errors="coerce")

# =============================
# BASIC CLEANING
# =============================

klaim = klaim.dropna(subset=["nomor_polis"])
klaim["nominal_klaim_yang_disetujui"] = klaim["nominal_klaim_yang_disetujui"].fillna(0)

df = klaim.merge(polis, on="nomor_polis", how="left")

# =============================
# MONTH AXIS (VERY IMPORTANT)
# =============================

MONTH_TYPE = "PAYMENT"   # <<< GANTI KE PAYMENT

if MONTH_TYPE == "SERVICE":
    month_col = "tanggal_pasien_masuk_rs"
elif MONTH_TYPE == "PAYMENT":
    month_col = "tanggal_pembayaran_klaim"
else:
    raise ValueError("MONTH_TYPE must be SERVICE or PAYMENT")

df = df.dropna(subset=[month_col]).copy()
df["year_month"] = df[month_col].dt.to_period("M")

# =============================
# MONTHLY AGGREGATION
# =============================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

print("Month type:", MONTH_TYPE)
print("Monthly shape:", monthly.shape)
print("Unique months:", monthly["year_month"].nunique())
print("\nSTAGE 1 v5 READY")


Month type: PAYMENT
Monthly shape: (24, 4)
Unique months: 24

STAGE 1 v5 READY


In [3]:
# ============================================================
# STAGE 1 v4 — USE PAYMENT DATE (CRITICAL TEST)
# ============================================================

df["year_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

print(monthly)

   year_month  frequency   total_claim
0     2024-01          8  1.283162e+08
1     2024-02         92  2.684171e+09
2     2024-03         97  3.809944e+09
3     2024-04        221  9.281203e+09
4     2024-05        233  1.103847e+10
5     2024-06        221  1.127720e+10
6     2024-07        205  1.159773e+10
7     2024-08        285  1.895989e+10
8     2024-09        250  1.484250e+10
9     2024-10        242  1.114198e+10
10    2024-11        365  1.740396e+10
11    2024-12        295  1.409901e+10
12    2025-01        293  1.697253e+10
13    2025-02        183  9.559585e+09
14    2025-03        234  1.494105e+10
15    2025-04        184  7.538943e+09
16    2025-05        201  9.628068e+09
17    2025-06        204  1.617766e+10
18    2025-07        272  1.862361e+10
19    2025-08        245  1.546896e+10
20    2025-09        197  1.041073e+10
21    2025-10         58  4.900102e+09
22    2025-11          3  1.356322e+08
23    2025-12          2  1.366003e+08


# TIME-SERIES DATASET ENGINEERING

In [4]:
# ============================================================
# STAGE 2 v3 — STABLE SEGMENT SHARE PANEL
# Plan-Level Only • Share-Based • Short Series Safe
# ============================================================

import numpy as np
import pandas as pd

# ============================================================
# ENSURE PLAN CODE EXISTS
# ============================================================

if "plan_code" not in df.columns:
    df["plan_code"] = "UNKNOWN"

df["plan_code"] = df["plan_code"].fillna("UNKNOWN")

# ============================================================
# BUILD MONTHLY PLAN-LEVEL PANEL
# ============================================================

seg_monthly = (
    df.groupby(["year_month", "plan_code"])
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values(["plan_code","year_month"])
      .reset_index(drop=True)
)

# ============================================================
# MERGE GLOBAL TOTAL (UNTUK SHARE)
# ============================================================

global_monthly = (
    df.groupby("year_month")
      .agg(total_global=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
)

seg_monthly = seg_monthly.merge(global_monthly, on="year_month", how="left")

# ============================================================
# SHARE OF TOTAL (STABLE TARGET)
# ============================================================

seg_monthly["share_total"] = (
    seg_monthly["total_claim"] /
    seg_monthly["total_global"].replace(0,np.nan)
)

# ============================================================
# SMOOTH SHARE (SHORT SERIES SAFE)
# ============================================================

seg_monthly["share_roll3"] = (
    seg_monthly.groupby("plan_code")["share_total"]
    .transform(lambda x: x.rolling(3, min_periods=1).mean())
)

# ============================================================
# LOG DOMAIN (OPTIONAL)
# ============================================================

seg_monthly["log_total"] = np.log1p(seg_monthly["total_claim"])
seg_monthly["log_freq"]  = np.log1p(seg_monthly["frequency"])

# ============================================================
# LAG FEATURES (SAFE)
# ============================================================

for col in ["log_total","log_freq","share_roll3"]:
    seg_monthly[f"{col}_lag1"] = \
        seg_monthly.groupby("plan_code")[col].shift(1)
    
    seg_monthly[f"{col}_lag2"] = \
        seg_monthly.groupby("plan_code")[col].shift(2)

# ============================================================
# DROP EARLY NA (MINIMAL LOSS)
# ============================================================

seg_model = seg_monthly.dropna().reset_index(drop=True)

# ============================================================
# FINAL CHECK
# ============================================================

print("SEGMENT PANEL SHAPE:", seg_model.shape)
print("Unique plan codes:", seg_model["plan_code"].nunique())
print("Months per plan (min):", 
      seg_model.groupby("plan_code")["year_month"].nunique().min())

print("\nSTAGE 2 v3 — STABLE SEGMENT SHARE PANEL READY")


SEGMENT PANEL SHAPE: (62, 15)
Unique plan codes: 3
Months per plan (min): 19

STAGE 2 v3 — STABLE SEGMENT SHARE PANEL READY


# MODEL DEVELOPMENT

In [5]:
# ============================================================
# STAGE 3 FINAL v2 — COHERENT ETS MODEL
# 5-STEP RECURSIVE • SERVICE DATE
# Frequency & Severity modeled
# Total = freq × sev
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ============================================================
# BUILD MONTHLY (SERVICE DATE)
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

# ============================================================
# SPLIT
# ============================================================

train = monthly.iloc[:-5].copy()
valid = monthly.iloc[-5:].copy()

sim_df = train.copy()

pred_total = []
pred_freq  = []
pred_sev   = []

# ============================================================
# RECURSIVE FORECAST
# ============================================================

for step in range(5):

    train_sim = sim_df.copy()

    # -------- FREQUENCY --------
    try:
        model_freq = ExponentialSmoothing(
            np.log1p(train_sim["frequency"]),
            trend="add",
            damped_trend=True,
            seasonal=None
        ).fit(optimized=True)

        freq_pred = np.expm1(model_freq.forecast(1).iloc[0])
    except:
        freq_pred = train_sim["frequency"].iloc[-1]

    freq_pred = max(freq_pred, 1)

    # -------- SEVERITY --------
    try:
        model_sev = ExponentialSmoothing(
            np.log1p(train_sim["severity"]),
            trend="add",
            damped_trend=True,
            seasonal=None
        ).fit(optimized=True)

        sev_pred = np.expm1(model_sev.forecast(1).iloc[0])
    except:
        sev_pred = train_sim["severity"].iloc[-1]

    # -------- COHERENT TOTAL --------
    total_pred = freq_pred * sev_pred

    pred_total.append(total_pred)
    pred_freq.append(freq_pred)
    pred_sev.append(sev_pred)

    new_row = {
        "year_month": None,
        "frequency": freq_pred,
        "total_claim": total_pred,
        "severity": sev_pred
    }

    sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

# ============================================================
# RESULTS
# ============================================================

print("\n==============================")
print("STAGE 3 v2 MAPE Frequency :", round(mape(valid["frequency"], pred_freq),4))
print("STAGE 3 v2 MAPE Total     :", round(mape(valid["total_claim"], pred_total),4))
print("STAGE 3 v2 MAPE Severity  :", round(mape(valid["severity"], pred_sev),4))
print("Estimated Score           :", round(np.mean([
    mape(valid["frequency"], pred_freq),
    mape(valid["total_claim"], pred_total),
    mape(valid["severity"], pred_sev)
]),4))
print("==============================")



STAGE 3 v2 MAPE Frequency : 5.9765
STAGE 3 v2 MAPE Total     : 6.9295
STAGE 3 v2 MAPE Severity  : 4.5584
Estimated Score           : 5.8215


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [6]:
# ============================================================
# STAGE 4 v2 — ADVANCED ETS OPTIMIZATION (TOTAL ONLY)
# Tune smoothing parameters directly
# ============================================================

!pip install -q optuna

import optuna
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

# ==============================
# BUILD MONTHLY
# ==============================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(total_claim=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["log_total"] = np.log1p(monthly["total_claim"])

train_full = monthly.iloc[:-4].copy()
valid_full = monthly.iloc[-4:].copy()

# ==============================
# OBJECTIVE
# ==============================

def objective(trial):

    trend_type = trial.suggest_categorical("trend", ["add", None])
    damped = trial.suggest_categorical("damped", [True, False])

    alpha = trial.suggest_float("alpha", 0.2, 0.95)
    beta  = trial.suggest_float("beta", 0.01, 0.5)
    phi   = trial.suggest_float("phi", 0.7, 0.99)

    sim_df = train_full.copy()
    preds = []

    for step in range(4):

        sub_train = sim_df.copy()

        try:
            model = ExponentialSmoothing(
                sub_train["log_total"],
                trend=trend_type,
                damped_trend=damped,
                seasonal=None,
                initialization_method="estimated"
            ).fit(
                smoothing_level=alpha,
                smoothing_trend=beta if trend_type=="add" else None,
                damping_trend=phi if damped else None,
                optimized=False
            )

            pred = np.expm1(model.forecast(1).iloc[0])

        except:
            pred = sub_train["total_claim"].iloc[-1]

        preds.append(pred)

        new_row = {
            "year_month": None,
            "total_claim": pred,
            "log_total": np.log1p(pred)
        }

        sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

    return mape(valid_full["total_claim"], preds)

# ==============================
# RUN OPTUNA
# ==============================

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=120)

print("\nBest Params:", study.best_params)
print("Best 4M TOTAL MAPE:", round(study.best_value*100,4), "%")

[I 2026-02-18 00:46:38,914] A new study created in memory with name: no-name-d62a3daf-81af-4e67-a43c-f86ebf5c2ba4
[I 2026-02-18 00:46:38,933] Trial 0 finished with value: 0.10291648767298633 and parameters: {'trend': None, 'damped': False, 'alpha': 0.20327872960331708, 'beta': 0.12719825388442946, 'phi': 0.8707818632235856}. Best is trial 0 with value: 0.10291648767298633.
[I 2026-02-18 00:46:38,948] Trial 1 finished with value: 0.3452768783473275 and parameters: {'trend': 'add', 'damped': False, 'alpha': 0.7368682852278516, 'beta': 0.4494598437153202, 'phi': 0.7544667329150798}. Best is trial 0 with value: 0.10291648767298633.
[I 2026-02-18 00:46:38,962] Trial 2 finished with value: 0.16943018641989543 and parameters: {'trend': None, 'damped': False, 'alpha': 0.7422830155566387, 'beta': 0.1458915765312748, 'phi': 0.7786741738758112}. Best is trial 0 with value: 0.10291648767298633.
[I 2026-02-18 00:46:38,978] Trial 3 finished with value: 0.1550560173424565 and parameters: {'trend': 'a


Best Params: {'trend': 'add', 'damped': True, 'alpha': 0.20137827227952154, 'beta': 0.18079869937629814, 'phi': 0.9861012138922342}
Best 4M TOTAL MAPE: 5.8604 %


# TEST PREDICTION & KAGGLE SUBMISSION

In [7]:
# ============================================================
# STAGE 5 — EXACT COPY FROM STAGE 3 (SERVICE DATE)
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# === BUILD MONTHLY IDENTICAL TO STAGE 3 ===
df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

# === PREPARE FUTURE MONTHS ===
sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

sim_df = monthly.copy()
predictions = {}

for period in future_periods:

    train_sim = sim_df.copy()

    # FREQUENCY
    model_freq = ExponentialSmoothing(
        np.log1p(train_sim["frequency"]),
        trend="add",
        damped_trend=True,
        seasonal=None
    ).fit(optimized=True)

    freq_pred = np.expm1(model_freq.forecast(1).iloc[0])
    freq_pred = max(freq_pred, 1)

    # SEVERITY
    model_sev = ExponentialSmoothing(
        np.log1p(train_sim["severity"]),
        trend="add",
        damped_trend=True,
        seasonal=None
    ).fit(optimized=True)

    sev_pred = np.expm1(model_sev.forecast(1).iloc[0])

    # TOTAL COHERENT
    total_pred = freq_pred * sev_pred

    new_row = {
        "year_month": period,
        "frequency": freq_pred,
        "total_claim": total_pred,
        "severity": sev_pred
    }

    sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

    key_total = f"{period.year}_{str(period.month).zfill(2)}_Total_Claim"
    key_freq  = f"{period.year}_{str(period.month).zfill(2)}_Claim_Frequency"
    key_sev   = f"{period.year}_{str(period.month).zfill(2)}_Claim_Severity"

    predictions[key_total] = total_pred
    predictions[key_freq]  = freq_pred
    predictions[key_sev]   = sev_pred

submission = sample_sub.copy()
submission["value"] = submission["id"].map(predictions)
submission = submission[["id","value"]]

submission.to_csv("submission.csv", index=False)

print("Submission created — EXACT STAGE 3 COPY")


Submission created — EXACT STAGE 3 COPY


In [8]:
print(submission.head(9)) ##11 persen 

                        id         value
0  2025_08_Claim_Frequency  2.330006e+02
1   2025_08_Claim_Severity  5.222999e+07
2      2025_08_Total_Claim  1.216962e+10
3  2025_09_Claim_Frequency  2.329167e+02
4   2025_09_Claim_Severity  5.220760e+07
5      2025_09_Total_Claim  1.216002e+10
6  2025_10_Claim_Frequency  2.328496e+02
7   2025_10_Claim_Severity  5.218970e+07
8      2025_10_Total_Claim  1.215235e+10
